In [ ]:
!pip install "tensorflow<2.11";
import numpy as np
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img


In [ ]:
"""import zipfile
import os

# Define the path to the zip file and the directory to unzip to
zip_file_path = 'clothing-dataset-small-master.zip'
unzip_dir = 'clothing-dataset-small-master'

# Create the directory if it doesn't exist
os.makedirs(unzip_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f"Extracted all files to {unzip_dir}") """


In [ ]:
path = './clothing-dataset-small/train/t-shirt'
name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
fullname = f'{path}/{name}'
load_img(fullname)

In [ ]:
img = load_img(fullname, target_size=(299, 299))

In [ ]:
x = np.array(img)
x

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
 
# weights = "imagenet" means we want to use pre-trained network that was trained on imagenet
 
model = Xception(
    weights="imagenet",
    input_shape=(299, 299, 3)
)

In [ ]:
X = np.array([x])
X.shape

In [ ]:
X = preprocess_input(X)
X[0]

In [ ]:
pred = model.predict(X)

In [ ]:
decode_predictions(pred)

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(150, 150),
    batch_size=32
)

Found 3068 images belonging to 10 classes.


In [18]:
X, y = next(train_ds)

In [20]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

Found 341 images belonging to 10 classes.


In [21]:
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)
 
base_model.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [22]:
inputs = keras.Input(shape=(150, 150, 3))
base = base_model(inputs)
outputs = base
model = keras.Model(inputs, outputs)
preds = model.predict(X)



1/1 [==============================] - 2s 2s/step


In [24]:
inputs = keras.Input(shape=(150, 150, 3))
base = base_model(inputs)
 
pooling = keras.layers.GlobalAveragePooling2D()
vectors = pooling(base)
outputs = vectors
 
model = keras.Model(inputs, outputs)
preds = model.predict(X)

1/1 [==============================] - 2s 2s/step


In [25]:
inputs = keras.Input(shape=(150, 150, 3))
 
base = base_model(inputs)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = vectors
model = keras.Model(inputs, outputs)
 
preds = model.predict(X)
preds.shape

1/1 [==============================] - 2s 2s/step


(32, 2048)

In [26]:
inputs = keras.Input(shape=(150, 150, 3))
 
base = base_model(inputs)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(10)(vectors)
model = keras.Model(inputs, outputs)
 
preds = model.predict(X)
preds.shape

1/1 [==============================] - 2s 2s/step


(32, 10)

In [27]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
 
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [29]:
# history = model.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5


96/96 [==============================] - 176s 2s/step - loss: 0.6754 - accuracy: 0.7982 - val_loss: 0.8228 - val_accuracy: 0.7889
Epoch 2/5
96/96 [==============================] - 175s 2s/step - loss: 0.3931 - accuracy: 0.8742 - val_loss: 0.9805 - val_accuracy: 0.8006
Epoch 3/5
96/96 [==============================] - 175s 2s/step - loss: 0.2758 - accuracy: 0.9035 - val_loss: 1.0177 - val_accuracy: 0.7918
Epoch 4/5
96/96 [==============================] - 175s 2s/step - loss: 0.1699 - accuracy: 0.9377 - val_loss: 0.8469 - val_accuracy: 0.8006
Epoch 5/5
96/96 [==============================] - 175s 2s/step - loss: 0.1226 - accuracy: 0.9570 - val_loss: 0.8888 - val_accuracy: 0.8006


In [30]:
model.save_weights('model_v1.h5', save_format='h5')
 
# Keras uses this template for saving files.
'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5'.format(epoch=3, val_accuracy=0.84)
 

'xception_v1_03_0.840.h5'

In [31]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [32]:
def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )
 
    base_model.trainable = False
 
    #########################################
 
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
     
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
     
    outputs = keras.layers.Dense(10)(inner)
     
    model = keras.Model(inputs, outputs)
     
    #########################################
 
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
     
    return model


In [33]:
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )
 
    base_model.trainable = False
 
    #########################################
 
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
     
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
     
    outputs = keras.layers.Dense(10)(drop)
     
    model = keras.Model(inputs, outputs)
     
    #########################################
 
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
     
    return model


In [34]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=10,
    height_shift_range=10,
    shear_range=10,
    zoom_range=0.1,
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=True,
)

In [35]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    vertical_flip=True,
)
 
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(150, 150),
    batch_size=32
)
 
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

Found 3068 images belonging to 10 classes.
Found 341 images belonging to 10 classes.


In [36]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100,
               droprate=0.5):
 
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(input_size, input_size, 3)
    )
 
    base_model.trainable = False
 
    #########################################
 
    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
     
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
     
    outputs = keras.layers.Dense(10)(drop)
     
    model = keras.Model(inputs, outputs)
     
    #########################################
 
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
     
    return model


In [37]:
input_size = 299
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)
 
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(input_size, input_size),
    batch_size=32
)
 
 
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
val_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(input_size, input_size),
    batch_size=32,
    shuffle=False
)

Found 3068 images belonging to 10 classes.
Found 341 images belonging to 10 classes.


In [38]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v4_1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [39]:
learning_rate = 0.0005
size = 100
droprate = 0.2
 
model = make_model(
    input_size=input_size,
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate
)

In [46]:
learning_rate = 0.0005
size = 100
droprate = 0.2
 
model = make_model(
    input_size=input_size,
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate
)
 
history = model.fit(train_ds, epochs=1, validation_data=val_ds,
                   callbacks=[checkpoint])

76/96 [======================>.......] - ETA: 2:18 - loss: 1.0878 - accuracy: 0.6499

KeyboardInterrupt: 

In [44]:
import tensorflow as tf
from tensorflow import keras
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input


In [41]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
test_ds = test_gen.flow_from_directory(
    './clothing-dataset-small/test',
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)

Found 372 images belonging to 10 classes.


In [45]:
model = keras.models.load_model('xception_v1_03_0.8651.h5')

OSError: No file or directory found at xception_v1_03_0.8651.h5